In [16]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [17]:
df = pd.read_csv("accident.csv")

In [18]:
df.fillna(df.mode().iloc[0], inplace=True)

In [19]:
label_cols = ["Gender", "Helmet_Used", "Seatbelt_Used"]
encoder = LabelEncoder()
for col in label_cols:
    df[col] = encoder.fit_transform(df[col])

In [20]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [22]:
param_grids = {
    "SVM": {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear']
    },
    "Gradient Boosting": {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5]
    },
    "XGBoost": {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5]
    }
}

In [23]:
models = {
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Support Vector Machine": SVC(),
    "Naive Bayes": GaussianNB()
}

In [24]:
scalers = {
    "L1 Normalization": Normalizer(norm="l1"),
    "L2 Normalization": Normalizer(norm="l2"),
    "Min-Max Scaling": MinMaxScaler(),
    "Standard Scaling": StandardScaler()
}

In [25]:
results_preprocessing = {}

for scale_name, scaler in scalers.items():
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    results_preprocessing[scale_name] = {}

    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        results_preprocessing[scale_name][name] = {
            "Accuracy": round(accuracy_score(y_test, y_pred), 3),
            "Precision": round(precision_score(y_test, y_pred, average='weighted'), 3),
            "Recall": round(recall_score(y_test, y_pred, average='weighted'), 3),
            "F1 Score": round(f1_score(y_test, y_pred, average='weighted'), 3)
        }

with open("./json/results_preprocessing.json", "w") as f:
    json.dump(results_preprocessing, f, indent=4)

/home/prabhuji/dev/labs/ad/report/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/prabhuji/dev/labs/ad/report/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/prabhuji/dev/labs/ad/report/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

In [26]:
results_no_tuning = {}
best_hyperparameters = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results_no_tuning[name] = {
        "Accuracy": round(accuracy_score(y_test, y_pred), 3),
        "Precision": round(precision_score(y_test, y_pred, average='weighted'), 3),
        "Recall": round(recall_score(y_test, y_pred, average='weighted'), 3),
        "F1 Score": round(f1_score(y_test, y_pred, average='weighted'), 3)
    }
    
with open("./json/results_no_tuning.json", "w") as f:
    json.dump(results_no_tuning, f, indent=4)

In [27]:
results_tuned = {}

for name, model in models.items():
    if name in param_grids:
        grid = GridSearchCV(model, param_grids[name], cv=10, scoring='accuracy', n_jobs=-1)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        best_params = grid.best_params_
    else:
        best_model = model
        best_model.fit(X_train, y_train)
        best_params = best_model.get_params()

    scores = cross_val_score(best_model, X_train, y_train, cv=10, scoring='accuracy')
    avg_accuracy = scores.mean()

    y_pred = best_model.predict(X_test)

    results_tuned[name] = {
        "Accuracy": round(accuracy_score(y_test, y_pred), 3),
        "Precision": round(precision_score(y_test, y_pred, average='weighted'), 3),
        "Recall": round(recall_score(y_test, y_pred, average='weighted'), 3),
        "F1 Score": round(f1_score(y_test, y_pred, average='weighted'), 3),
        "Cross-Validation Accuracy (10-Fold)": round(avg_accuracy, 3)
    }
    best_hyperparameters[name] = {key: value for key, value in best_params.items()}

with open("./json/results_tuned.json", "w") as f:
    json.dump(results_tuned, f, indent=4)

with open("./json/best_hyperparameters.json", "w") as f:
    json.dump(best_hyperparameters, f, indent=4)

In [28]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

In [29]:
results_pca = {}

for model_name, model in models.items():
    model.fit(X_pca, y)
    y_pred = model.predict(X_pca)
    
    results_pca[model_name] = {
        "Accuracy": round(accuracy_score(y, y_pred), 3),
        "Precision": round(precision_score(y, y_pred, average='macro'), 3),
        "Recall": round(recall_score(y, y_pred, average='macro'), 3),
        "F1 Score": round(f1_score(y, y_pred, average='macro'), 3)
    }

with open("./json/results_pca.json", "w") as json_file:
    json.dump(results_pca, json_file, indent=4)

In [30]:
for model_name, model in models.items():
    with open(f"{model_name.replace(' ', '_').lower()}.pkl", "wb") as f:
        pickle.dump(model, f)

In [32]:
X.head()

,Age,Gender,Speed_of_Impact,Helmet_Used,Seatbelt_Used
0,56,0,27.0,0,0
1,69,0,46.0,0,1
2,46,1,46.0,1,1
3,32,1,117.0,0,1
4,60,0,40.0,1,1
